In [14]:
#import modules
import pandas as pd 
import numpy as np
import string
import re
import time
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

#mask warnings
import warnings
#warnings.filterwarnings("ignore")

#import our functions
from functions import *

#get path
import os
path = os.getcwd()

In [2]:
#df des individus 
df = pd.read_csv(f"{path}/20ksample.csv", encoding='latin-1', low_memory=False)
#df = pd.read_csv(f"{path}/100ksample.csv", sep=";", encoding='latin-1', low_memory=False)

#clean df and save it
df = df.rename(columns={'LIBELLE_EMPLOI_GRADE': 'libelle', 'EMPLOI_STATUTAIRE':'code'})
print(df.isna().sum()/len(df)*100, "%")
df.libelle = df.libelle.astype('category')
df['libelle'] = df['libelle'].apply(clean_libelle)
df.to_csv("clean20K.csv", index=False)

#subset 2011
df = pd.read_csv(f"{path}/clean20k.csv", encoding='latin-1', low_memory=False)
df.libelle = df.libelle.astype("str")
df['date_fin'] = pd.to_datetime(df['date_fin'], format='%d/%m/%Y')
df['date_debut'] = pd.to_datetime(df['date_debut'], format='%d/%m/%Y')

ID_CARRIERE          0.000000
date_debut           0.000000
date_fin             0.000000
ETAT                 0.000000
CATEGORIE_EMPLOI     0.020218
QUALITE              0.027084
STATUT_AGENT         0.026512
INDICE_BRUT         20.959779
code                54.151396
libelle              3.948978
Unnamed: 10         99.528026
Unnamed: 11         99.997330
dtype: float64 %


In [3]:
#PRINT DES EXEMPLES DU NETTOYAGE DE LIBELLÉS
n = 1000
df_random = df.sample(n)
#for i in range(n):
#    print(f"{df_random.libelle.iloc[i]}\n->{df_random.libelle2.iloc[i]}\n")

<br>
<br>
<br>
<br>

# Correction des libellés

##### Parmi l'ensemble des individus, si un code un présent, alors on va chercher dans la nomenclature le libelle correspondant. Cela a pour objectif d'augmenter le nombre d'individus avec un libellé standardisé.

In [4]:
#open nomenclature
nomenclature = pd.read_csv(f"{path}/nomenclature.csv")

#rename colonnes et convert to string
nomenclature = nomenclature.rename(columns={'libelle_NEG_rempl': 'libelle', 'code_NETH_rempl':'code'})
nomenclature.libelle = nomenclature.libelle.astype("str")

#clean libellés de la nomenclature
nomenclature['libelle'] = nomenclature['libelle'].apply(clean_libelle)

#faire un set de la nomenclature
set_nomenclature = set(nomenclature.libelle)

In [5]:
#remplacer libelle (si code, alors mettre le libelle standardisé)
df = remplacer_libelles(df, nomenclature, "code", "libelle")

<br>
<br>
<br>
<br>

# Approximation des libellés non-standard

##### Parmi les libellés non-standardisés restant, on souhaite, à remplacer par un libellé standardisé. Pour cela, on doit calculer une similarité entre les libellés standardisés et non-standardisés. Pour cela il faut d'abord vectorizer les libellés, nous avons utilisés différentes méthodes :
- Count vectorizer 
- Tfidf vectorizer 
- Hashing vectorizer

##### Méthode de similarité testée :
- [similarité cosinus](https://fr.wikipedia.org/wiki/Similarité_cosinus) (cosinus de l'angle entre 2 vecteurs de dimensions n)

In [6]:
#random sample from the df with the observations
n = 3000
non_standard_labels = list(df.libelle.sample(n))

#definir libellés standards et non standards
non_standard_labels = list(df.libelle)
standard_labels = list(set_nomenclature)

In [7]:
#define a vectorizer
vectorizer1 = CountVectorizer()
vectorizer2 = TfidfVectorizer()
vectorizer3 = HashingVectorizer()
X_non_standard = vectorizer3.fit_transform(non_standard_labels)
X_standard = vectorizer3.transform(standard_labels)

In [8]:
#compute cosine similarity for each obs for each libelle standardisé
start = time.time()

#calcul de la matrice de cosinus similarité
similarity_matrix = cosine_similarity(X_non_standard, X_standard)

#initialiser la liste des nouveaux libellés
most_similar_labels = []
for i in tqdm(range(len(non_standard_labels))):
    
    #si nan, alors on passe
    if non_standard_labels[i] == "nan":
        most_similar_labels.append(np.nan)
    
    #sinon, on prend la plus grande similarité
    else:
        most_similar_index = similarity_matrix[i].argmax()
        most_similar_label = standard_labels[most_similar_index]
        #print(f"{non_standard_labels[i]}\n-> {most_similar_label}\n\n")
        most_similar_labels.append(most_similar_label)
    
end = time.time()
print(f"Time (sec): {round(end - start,2)}")

#ajouter les nouveaux labels
df.libelle = most_similar_labels

100%|██████████| 1048575/1048575 [00:55<00:00, 18945.63it/s]

Time (sec): 193.31


In [9]:
#on enregistre le nouveau df 20k
df.to_csv("clean20K.csv", index=False)

<br>
<br>
<br>
<br>

# Deduction des codes grâce aux nouveaux libellés

In [15]:
#ouvrir le df et afficher nan
df = pd.read_csv("clean20K.csv", low_memory=False)
print(df.isna().sum()/len(df)*100)

#ajouter les codes à partir des libellés
na1 = df.isna().sum().sum()
df = remplacer_code(df, nomenclature, "code", "libelle")
na2 = df.isna().sum().sum()

#print nan enlevés
print(f"Nombre de Nan enlevés: {na1-na2}")

ID_CARRIERE          0.000000
date_debut           0.000000
date_fin             0.000000
ETAT                 0.000000
CATEGORIE_EMPLOI     0.020218
QUALITE              0.027084
STATUT_AGENT         0.026512
INDICE_BRUT         20.959779
code                54.151396
libelle              5.308538
Unnamed: 10         99.528026
Unnamed: 11         99.997330
dtype: float64


AttributeError: 'Series' object has no attribute 'fill'

In [ ]:
df.to_csv("clean20K.csv", index=False)

<br>
<br>
<br>
<br>

# Remplissage indice brut

In [ ]:
df = pd.read_csv("clean20K.csv", low_memory=False)
df.isna().sum()/len(df)*100